In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
tests_by_zip = pd.read_csv('data/accum-nychealth-tests-by-zcta.csv')
tests_by_zip = tests_by_zip.dropna()
tests_by_zip['ZipCode'] = tests_by_zip['MODZCTA'].astype('int')

In [ ]:
ny_zips = pd.read_csv('data/NY_Census_Zip.csv')
ny_zips['ZipCode'] = ny_zips['NAME'].replace(to_replace=r'ZCTA5 (\d+), New York', value=r'\1', regex=True)
ny_zips = ny_zips[ny_zips['ZipCode'].apply(lambda x: x.isnumeric())]
ny_zips['ZipCode'] = ny_zips['ZipCode'].astype('int')

In [ ]:
days = 5
df = tests_by_zip.merge(ny_zips, on='ZipCode')
df = df[['GEO_ID', 'ZipCode', 'P001001', 'Total', 'Positive', 'Timestamp']]
df.columns = ['GEO ID', 'Zip Code', 'Population', 'Total Tests', 'Positive CVD', 'Timestamp']

df['Timestamp'] = pd.to_datetime(df['Timestamp'])

df['Population'] = df['Population'].astype('int')
df['Incidence Rate'] = df['Positive CVD'] / df['Population'] * 100

new_df = pd.DataFrame()
for by, group in df.groupby(['Zip Code']):
    if 11697 == by:
        # Remove duplicate data for zip code 11697.
        duplicate_key = (group['Timestamp'] == '2020-04-10 17:41:39') & (group['Total Tests'] == 105)
        group = group[~duplicate_key]
    group = group.set_index('Timestamp', verify_integrity=True)
    group = group.sort_values(by='Timestamp')
    group.loc[:, str(days) + ' Day Mean Incidence Rate'] = group['Incidence Rate'].rolling(str(days) + 'd').mean()
    new_df = pd.concat([new_df, group])
df = new_df

In [ ]:
demographic_df = pd.read_csv('data/Demographic_Statistics_By_Zip_Code.csv')
demographic_df['JURISDICTION NAME'].unique()
demographic_df = demographic_df.rename(columns={'JURISDICTION NAME': 'Zip Code'})
demographic_df['Zip Code'] = demographic_df['Zip Code'].astype('int')

df = df.merge(demographic_df, on='Zip Code')

In [ ]:
df.to_csv('5_Day_Mean_Incidence_Rate_Per_NY_Zip_Code.csv')

In [ ]:
df.columns